In [1]:
import numpy as np
import pandas as pd
import main
las_data = pd.read_csv(main.csv_las_out_full_path, delimiter=';')
# удаляем скважины, по которым нет водонасыщенности
well_name_list = las_data['well_name'].value_counts().index.tolist()
for well_name in well_name_list:
    well_data = las_data[las_data['well_name'] == well_name]
    if np.isnan(well_data['kvo'].mean()) or np.isnan(well_data['rp'].mean()) or np.isnan(well_data['kgl'].mean()) or np.isnan(well_data['aps'].mean()):
        # axis  = 0, удаляем столбцы
        las_data = las_data.drop(las_data[las_data['well_name'] == well_name].index, axis = 0)

In [2]:
# считает среднее значение параметра по высоте с пропуском nan
def calc_mean_value_by_h(h_arr, val_arr):
    hv_sum = 0.
    h_sum = 0.
    prev_nan_val = False
    for h, val in zip(h_arr, val_arr):
        if not(np.isnan(h)) and not(np.isnan(val)):
            h_sum += h
            hv_sum += h * val

    # print(h_sum, hv_sum)
    return hv_sum/h_sum

In [3]:
# считает среднее значение параметра по высоте с пропуском nan
def calc_mean_value(val_arr):
    sum = 0.
    n = 0
    for val in val_arr:
        if not np.isnan(val):
            sum += val
            n = n + 1

    # print(h_sum, hv_sum)
    if (n == 0):
        n = 1
    return sum/n

In [7]:
las_data = las_data[las_data['satur']>0]
las_data.head()

,well_name,STRT,STOP,DEPT,aps,rp,kp,kgl,kpr,kvo,kng,lit,satur,UWI
1182,224st4,2729.2,2929.4,2847.4,0.99,5.1,0.171,NaN,19.8,NaN,0.418,1.0,8.0,314_103ST2
1183,224st4,2729.2,2929.4,2847.5,0.99,5.1,0.171,NaN,19.8,NaN,0.418,1.0,8.0,314_103ST2
1184,224st4,2729.2,2929.4,2847.6,0.99,5.1,0.171,NaN,19.8,NaN,0.418,1.0,8.0,314_103ST2
1185,224st4,2729.2,2929.4,2847.7,0.99,5.1,0.171,NaN,19.8,NaN,0.418,1.0,8.0,314_103ST2
1186,224st4,2729.2,2929.4,2847.8,0.99,5.1,0.171,NaN,19.8,NaN,0.418,1.0,8.0,314_103ST2


In [8]:
# получим нефтенасыщенность
las_data['kno'] = 1 - las_data['kvo']

In [95]:
# переведем данные с 0 в 1 
from sklearn.preprocessing import MinMaxScaler
# уберем Nan
#las_data = las_data.ffill()
# las_data_norm=las_data.copy()
# las_data_norm['aps'] = MinMaxScaler().fit_transform(las_data['aps'])
# # las_data_norm['rp'] = MinMaxScaler().fit_transform(las_data['rp'])
# las_data_norm['kp'] = MinMaxScaler().fit_transform(las_data['kp'])
# las_data_norm['kgl'] = MinMaxScaler().fit_transform(las_data['kgl'])
# las_data_norm['kpr'] = MinMaxScaler().fit_transform(las_data['kpr'])
# las_data_norm['kvo'] = MinMaxScaler().fit_transform(las_data['kvo'])
# las_data_norm['kng'] = MinMaxScaler().fit_transform(las_data['kng'])
# # Применить нормализацию к данным
# las_data=las_data_norm

1521.0

In [11]:
# заполним данные по скважинам
well_name_list = las_data['well_name'].value_counts().index.tolist()
data_well_for_analize = pd.DataFrame() 
first = True
kno_list = []
kgl_list = []
rp_list = []
aps_list = []
kp_list = []
kpr_list = []
kng_list = []
satur_list = []
uwi_name_list = []
kvo_list = []
# kgl_std_list = []
# kgl_p25_list = []
# kgl_p50_list = []
# kgl_p75_list = []

for well_name in well_name_list:
    # print(well_name)
    data_well_list = []
    # Выделяем даные для одной скважины
    data_well = las_data[las_data['well_name'] == well_name]
    uwi_name_list.append(data_well['UWI'][data_well['UWI'].index[0]])
    # kno_list.append(calc_mean_value_by_h(data_well['DEPT'],data_well['kno']))
    # kgl_list.append(calc_mean_value_by_h(data_well['DEPT'],data_well['kgl']))
    # rp_list.append(calc_mean_value_by_h(data_well['DEPT'],data_well['rp']))
    # aps_list.append(calc_mean_value_by_h(data_well['DEPT'],data_well['aps']))
    # kp_list.append(calc_mean_value_by_h(data_well['DEPT'],data_well['kp']))
    # kpr_list.append(calc_mean_value_by_h(data_well['DEPT'],data_well['kpr']))
    # kng_list.append(calc_mean_value_by_h(data_well['DEPT'],data_well['kng']))
    # satur_list.append(calc_mean_value_by_h(data_well['DEPT'],data_well['satur']))
    
    kno_list.append(calc_mean_value(data_well['kno']))
    kgl_list.append(calc_mean_value(data_well['kgl']))
    rp_list.append(calc_mean_value(data_well['rp']))
    aps_list.append(calc_mean_value(data_well['aps']))
    kp_list.append(calc_mean_value(data_well['kp']))
    kpr_list.append(calc_mean_value(data_well['kpr']))
    kng_list.append(calc_mean_value(data_well['kng']))
    satur_list.append(calc_mean_value(data_well['satur']))
    kvo_list.append(calc_mean_value(data_well['kvo']))

    # 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'    
    # describe_val = data_well.describe()
    # kgl_std_list = describe_val['kgl']['std']
    # kgl_p25_list = describe_val['kgl']['25%']  
    # kgl_p50_list = describe_val['kgl']['50%']
    # kgl_p75_list = describe_val['kgl']['75%']
        

data_well_for_analize = pd.DataFrame({'well_name':well_name_list, 'UWI':uwi_name_list, 'kno':kno_list,
                                      'kgl':kgl_list, 'rp':rp_list, 
                                      'aps':aps_list, 'kp':kp_list, 'kpr':kpr_list,
                                      'kng':kng_list, 'satur':satur_list, 'kvo':kvo_list}
                                      )
# 'kgl_std':kgl_std_list, 'kgl_25':kgl_p25_list,
#                                       'kgl_50':kgl_p50_list, 'kgl_75':kgl_p75_list}

data_well_for_analize.head()

,UWI,aps,kgl,kng,kno,kp,kpr,kvo,rp,satur,well_name
0,314_2R,0.689105,0.178975,0.008030,0.617446,0.262150,298.548713,0.382554,3.910958,2.283684,4r
1,314_431PL,0.764744,0.177158,0.008719,0.638302,0.252416,106.838975,0.361698,3.598123,2.087578,643pl
2,314_342,0.687039,0.117067,0.053140,0.611152,0.197540,45.104132,0.388848,5.520386,2.542700,554
3,314_246,0.730486,0.115063,0.101729,0.478072,0.164370,8.436575,0.521928,6.312685,2.718816,458
4,314_381,0.687716,0.122134,0.187884,0.569034,0.176849,31.133190,0.430966,6.323276,3.887931,593


In [12]:
data_well_for_analize['kno'].describe()

count    175.000000
mean       0.514140
std        0.077992
min        0.104318
25%        0.487490
50%        0.532195
75%        0.563342
max        0.638302
Name: kno, dtype: float64

In [13]:
header_data = pd.read_csv(main.data_dir + "\\Well_Header.csv", delimiter='\t')
header_data.head()
header_data = header_data.rename(columns={'WellName': main.keys_dict[main.kid_well_dop_id]})
del header_data['KB']
del header_data['TopDepth']
del header_data['BottomDepth']
del header_data['Symbol']
del header_data['Rig']
del header_data['Field']
header_data.head()

,UWI,X-Coord,Y-Coord
0,314_103ST2,354719.52,6470051.42
1,314_107,354693.18,6470164.36
2,314_121,354433.25,6468384.38
3,314_125PL,354717.25,6470061.16
4,314_134,354478.44,6468306.61


In [14]:
# загружаем wc
water_content_data = pd.read_csv(main.data_dir + "\\water_content.csv", delimiter=';')
water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'wc'})
water_content_data = water_content_data.rename(columns={'Дата запуска': 'date_start'})
water_content_data.head()

,well_name,date_start,wc
0,380,04.07.1905,0.49
1,379,01.12.2006,0.30
2,370,01.02.2007,0.13
3,417,01.03.2007,0.08
4,389,01.08.2007,0.22


In [16]:
# del water_content_data['Дата запуска']
data_for_analize = pd.merge(data_well_for_analize, water_content_data, on=main.keys_dict[main.kid_well])
data_for_analize = pd.merge(data_for_analize, header_data, on=main.keys_dict[main.kid_well_dop_id])
data_for_analize.head()

,UWI,aps,kgl,kng,kno,kp,kpr,kvo,rp,satur,well_name,date_start,wc,X-Coord,Y-Coord
0,314_342,0.687039,0.117067,0.053140,0.611152,0.197540,45.104132,0.388848,5.520386,2.542700,554,01.06.2008,0.08,351657.56,6464689.29
1,314_246,0.730486,0.115063,0.101729,0.478072,0.164370,8.436575,0.521928,6.312685,2.718816,458,01.01.2016,0.42,349776.29,6463659.09
2,314_381,0.687716,0.122134,0.187884,0.569034,0.176849,31.133190,0.430966,6.323276,3.887931,593,01.04.2010,0.25,351543.76,6464555.85
3,314_328,0.607397,0.138881,0.158237,0.529251,0.134452,2.732420,0.470749,6.518721,3.753425,530,01.08.2009,0.20,352508.25,6467232.79
4,314_360,0.729539,0.116272,0.121862,0.587687,0.161848,20.141014,0.412313,6.454378,3.354839,572,01.08.2010,0.36,349941.96,6464113.27


In [17]:
#  сохраняем данные
data_for_analize.to_csv(main.data_dir + '\\data_for_analize.csv', sep=';', index=False)

In [30]:
# оставлеям только данные для x и y отдельно
x = data_for_analize.copy()
y = data_for_analize['wc']
well_names = data_for_analize['well_name']
# удаляем неиспользуеиые для обучения колонки
x.drop(['wc', 'well_name', 'date_start', 'UWI'], axis=1, inplace=True)
# округлим вектор ответов
x.head(1)

,aps,kgl,kng,kno,kp,kpr,kvo,rp,satur,X-Coord,Y-Coord
0,0.687039,0.117067,0.05314,0.611152,0.19754,45.104132,0.388848,5.520386,2.5427,351657.56,6464689.29


In [31]:
y_r = int(10) * round(y,1)
# y_r = []
# for val in y:
#     y_r.append(int(10*val))

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score

x_train, x_hold, y_train, y_hold = train_test_split(x.values, y_r, test_size=0.3, random_state=17)

In [33]:
tree = DecisionTreeClassifier(max_depth=20, random_state=17)
tree.fit(x_train, y_train)
tree_pred = tree.predict(x_hold)
accuracy_score(y_hold, tree_pred)

0.16666666666666666

In [34]:
from sklearn.model_selection import GridSearchCV, cross_val_score
count_vals = x.shape[1]
print(count_vals)
tree_params = {'max_depth': range(1,30), 'max_features': range(1, count_vals)}
tree_params = {'max_depth': range(1,30)}
tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, verbose=True)
tree_grid.fit(x_train, y_train)

11
Fitting 5 folds for each of 29 candidates, totalling 145 fits


D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[Parallel(n_jobs=-1)]: Done 145 out of 145 | elapsed:    3.8s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=17, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': range(1, 30)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=True)

In [25]:
tree_grid.best_params_

{'max_depth': 1}

In [35]:
tree_grid.best_score_

0.33333333333333331

In [36]:
accuracy_score(y_hold, tree_grid.predict(x_hold))

0.21428571428571427

In [45]:
from sklearn.ensemble import GradientBoostingClassifier
gboost = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.01,
                                       max_depth=2, random_state=100)
gboost.fit(x_train, y_train)
gboost_pred = gboost.predict(x_hold)
accuracy_score(y_hold, gboost_pred)

0.14285714285714285

In [57]:
from sklearn.grid_search import GridSearchCV

param_grid = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
              'max_depth': [2, 4, 6],
              'min_samples_leaf': [3, 5, 9, 17],
              #'max_features': range(2, len(x_train[0]))
              # 'max_features': [1.0, 0.3, 0.1] ## not possible in our example (only 1 fx)
              }

est = GradientBoostingRegressor(n_estimators=3000)
# this may take some minutes
gs_cv = GridSearchCV(est, param_grid, n_jobs=4).fit(x_train, y_train)

# best hyperparameter setting
gs_cv.best_params_
gs_cv.best_score_